# stko and stk

by: Andrew Tarzia

# Imports 

In [2]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

from rdkit import Chem 
from rdkit.Chem import AllChem as rdkit
from collections import defaultdict
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdDistGeom
IPythonConsole.ipython_3d = True

import py3Dmol
from IPython.display import Image
import matplotlib.pyplot as plt
import subprocess
import time
import stk
import stko
import spindry as spd
%matplotlib inline

In [3]:
def show_stk_mol(stk_mol):
    data = rdkit.MolToMolBlock(stk_mol.to_rdkit_mol())
    p = py3Dmol.view(
        data=data,
        style={'stick':{'colorscheme':'cyanCarbon'}}, 
        width=400,
        height=400,
    )
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    p.show()

# Working

In [4]:
bb1 = stk.BuildingBlock('NCCN', [stk.PrimaryAminoFactory()])
bb2 = stk.BuildingBlock(
    smiles='O=CC(C=O)C=O',
    functional_groups=[stk.AldehydeFactory()],
)
cage1 = stk.ConstructedMolecule(
    topology_graph=stk.cage.FourPlusSix((bb1, bb2)),
)

In [5]:
show_stk_mol(cage1)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
broken_bonds_by_id = []
disconnectors = []
for bi in cage1.get_bond_infos():
    if bi.get_building_block() is None:
        a1id = bi.get_bond().get_atom1().get_id()
        a2id = bi.get_bond().get_atom2().get_id()
        broken_bonds_by_id.append(sorted((a1id, a2id)))
        disconnectors.extend((a1id, a2id))

In [7]:
print(broken_bonds_by_id)
print(disconnectors)
print('--')

[[0, 32], [8, 35], [2, 40], [16, 43], [3, 48], [24, 51], [11, 56], [18, 59], [10, 64], [27, 67], [19, 72], [26, 75]]
[0, 32, 8, 35, 2, 40, 16, 43, 3, 48, 24, 51, 11, 56, 18, 59, 10, 64, 27, 67, 19, 72, 26, 75]
--


In [8]:
new_topology_graph = stko.TopologyExtractor()
tg_info = new_topology_graph.extract_topology(
    molecule=cage1,
    broken_bonds_by_id=broken_bonds_by_id,
    disconnectors=set(disconnectors),
)

In [9]:
import os
os.mkdir('tg_output')

In [10]:
print(tg_info.get_vertex_positions())
print(tg_info.get_connectivities())
print(tg_info.get_edge_pairs())

{0: array([0.0952583 , 0.05782588, 7.70819073]), 1: array([-6.26032559, -3.61722913, -2.67443437]), 2: array([ 6.26277511, -3.61298643, -2.67443437]), 3: array([-2.44952914e-03,  7.23021556e+00, -2.67443437e+00]), 4: array([-4.70844968, -2.71842469,  3.85903949]), 5: array([ 4.71605668, -2.72476131,  3.85903947]), 6: array([-0.00924248,  5.44294872,  3.85903889]), 7: array([-0.01403689, -5.46061179, -3.84785617]), 8: array([-4.73393618,  2.72180559, -3.84785617]), 9: array([ 4.73604697,  2.71814959, -3.84785617])}
{0: 3, 1: 3, 2: 3, 3: 3, 4: 2, 5: 2, 6: 2, 7: 2, 8: 2, 9: 2}
[(0, 4), (1, 4), (0, 5), (2, 5), (0, 6), (3, 6), (1, 7), (2, 7), (1, 8), (3, 8), (2, 9), (3, 9)]


In [11]:
cage1.write('tg_output/tg_cage.mol')
tg_info.write('tg_output/tg_info.pdb')